# Hypothesis Testing

In [1]:
import warnings

import matplotlib.pyplot as plt
import torch

import confounding_robust_inference as cri

# setup toy policy and data
_ = torch.random.manual_seed(0)
TORCH_DTYPE = cri.utils.types.get_default_torch_dtype()
beta = torch.as_tensor([1, 0.5, -0.5, 0, 0], dtype=TORCH_DTYPE)
policy = cri.policies.LogisticPolicy(beta)
data = cri.data.SyntheticDataBinary()
Y, T, X, _, p_t, _ = data.sample(1000)

In [2]:
# We can also compute the theoretical robust policy value by Monte Carlo sampling with the true data distribution:
true_lower_bound = data.evaluate_policy_lower_bound(policy, Gamma=1.5, n_mc=100000)
ipw = cri.estimators.IPWEstimator().fit(Y, T, X, p_t, policy).predict()
true_ipw = data.evaluate_policy(policy, n_mc=100000)
bias = true_ipw - ipw

In [4]:
est = cri.estimators.KCMCEstimator("Tan_box", Gamma=1.5, D=30)
est.fit(Y, T, X, p_t, policy)

In [5]:
est.predict_ci()

(tensor(3.0853, dtype=torch.float64), tensor(4.1423, dtype=torch.float64))

In [6]:
est.predict_ci(consider_second_order=True)

/home/kei/Desktop/master-thesis/confounding_robust_inference/estimators/kcmc.py:123: UserWarning: Current implementation of the asymptotics are known to be unstable, because of the difficulty of estimating the covariance and Hessian matrices involved. Please use the features with caution.
  warnings.warn(


(tensor(3.0063, dtype=torch.float64), tensor(4.0813, dtype=torch.float64))